In [2]:
import pandas as pd
from pysnptools.snpreader import Bed
snp_on_disk = Bed(
    "/data/cotton/zhenpingliu/LZP_fiberFullPopulationRNAseq/Genotype/Trait_376.bed",
    count_A1=False)
snpdata1 = snp_on_disk.read()
SNP_pos = pd.DataFrame(snpdata1.pos, index=snpdata1.sid)


In [23]:
#--------------------------------------------------------------
#TODO: 获取GWAS SNP在两各亲本材料中的基因型数据
#? 获取50kb范围内与lead SNP连锁的位点基因型
#--------------------------------------------------------------
GWAS_site=pd.read_csv("./GWAS_site.txt",header=0,index_col=None,sep="\s+")

In [40]:
ChromDict={
    'Ghir_A01':1,'Ghir_A02':2,'Ghir_A03':3,'Ghir_A04':4,'Ghir_A05':5,'Ghir_A06':6,
    'Ghir_A07':7,'Ghir_A08':8,'Ghir_A09':9,'Ghir_A10':10,'Ghir_A11':11,'Ghir_A12':12,
    'Ghir_A13':13,
    'Ghir_D01':14,'Ghir_D02':15,'Ghir_D03':16,'Ghir_D04':17,'Ghir_D05':18,'Ghir_D06':19,
    'Ghir_D07':20,'Ghir_D08':21,'Ghir_D09':22,'Ghir_D10':23,'Ghir_D11':24,'Ghir_D12':25,
    'Ghir_D13':26,
}
ChromDict2={
    'A01':1,'A02':2,'A03':3,'A04':4,'A05':5,'A06':6,
    'A07':7,'A08':8,'A09':9,'A10':10,'A11':11,'A12':12,
    'A13':13,
    'D01':14,'D02':15,'D03':16,'D04':17,'D05':18,'D06':19,
    'D07':20,'D08':21,'D09':22,'D10':23,'D11':24,'D12':25,
    'D13':26,
}

In [61]:
SNPlist=[]
for Chrom,leadSNP in GWAS_site[['Chr','Lead-SNP']].values:
    SNPlist.append(
        SNP_pos.loc[(SNP_pos[0]==ChromDict2.get(Chrom))&(SNP_pos[2]==int("".join(leadSNP.split(","))))].index[0]
    )
GWAS_site['SNP']=SNPlist
GWAS_site.to_csv("./GWAS_site.txt",header=True,index=False,sep="\t")

In [84]:
TWAS_site=pd.read_csv("./TWAS_site.txt",header=0,index_col=None,sep="\s+")
TWAS_site=TWAS_site.drop_duplicates()
TWAS_site['SNP']=TWAS_site.apply(
    lambda x:SNP_pos.loc[(SNP_pos[0]==int(x['GWAS'].split("_")[0]))&(SNP_pos[2]==int(x['GWAS'].split("_")[1]))].index[0],
    axis=1
)
TWAS_site.to_csv("TWAS_gene_site.txt",header=True,index=False,sep="\t")

In [99]:
#-------------------------------------------------------------
#TODO: 提取每个lead sNP的SNP cluster
#-------------------------------------------------------------
ld_SNPs=pd.read_csv("leadSNP.ld",header=0,index_col=None,sep="\s+")
leadSNP=pd.read_csv("All_leadSNP.txt",header=None,index_col=None,sep="\t")
out=[]
for snp in leadSNP[0].values:
    SNPlist=ld_SNPs.loc[ld_SNPs['SNP_A']==snp]
    out.append(
        (snp,"-".join(SNPlist['SNP_B']))
    )
pd.DataFrame(out,columns=['leadSNP','SNPCluster']).to_csv("leadSNP_cluster.txt",header=True,index=False,sep="\t")    

In [4]:
import pysam
tabObject=pysam.TabixFile("/data/cotton/zhenpingliu/LZP_fiberFullPopulationRNAseq/Genotype/All_376_.filter2.Chrs.hmp.gz")
Samples=tabObject.header[0].split("\t")[11:]
tmp=pd.DataFrame(zip(range(0,376),Samples))
tmp.loc[tmp[1].isin(['S321','S326'])]

,0,1
22,22,S326
98,98,S321


In [193]:
#----------------------------------------------------------------------
#* 分析GWAS QTL随机挑选50个SNP，统计这50个SNP在两个亲本材料中的差异情况
#* ZY220对应 RNA编号为S321，对应的列数为98
#* ZY243对应 RNA编号为S326，对应的列数为22
#----------------------------------------------------------------------
leadSNPCluster=pd.read_csv("./leadSNP_cluster.txt",index_col=0,header=0,sep="\t")
out=[]
def getGenotypeSame(x):
    leadChrom=SNP_pos.loc[x].iloc[0]
    leadStart=SNP_pos.loc[x].iloc[2]
    SNPData=list(tabObject.fetch(reference=int(leadChrom),start=leadStart,end=leadStart+1))[0].split("\t")[11:]
    if SNPData[22]!=SNPData[98]:
        return 1
    else:
        #* 两各亲本在SNP的基因型相同
        return 0
    
#* 对所有的数据进行分析
for leadSNP  in TWAS_site['SNP']:
    leadSNPcode=getGenotypeSame(leadSNP)
    SNPcluster=leadSNPCluster.loc[leadSNP][0].split("-")
    SNPcluster_array=[]
    if len(SNPcluster)>=20:
        for snp in  SNPcluster[0:20]:
            SNPcluster_array.append(
                getGenotypeSame(snp)
            ) 
    else:
        #* 少于20个ld SNPs
        for snp in  SNPcluster:
            SNPcluster_array.append(
                getGenotypeSame(snp)
            )
        SNPcluster_array=SNPcluster_array+[-1]*(20-len(SNPcluster))
    
    out.append(
        [leadSNPcode]+SNPcluster_array
    )

In [191]:
GWAS_conserved = pd.DataFrame(out)
GWAS_conserved.to_csv("GWAS_conserved.txt", header=False,
                      index=False, sep="\t")


In [194]:
TWAS_conserved = pd.DataFrame(out)
TWAS_conserved.to_csv("TWAS_conserved.txt", header=False,
                      index=False, sep="\t")
